# **Prerequisites**

In [ ]:
!sudo sh -c 'echo "deb [arch=amd64 signed-by=/usr/share/keyrings/gpg-pub-moritzbunkus.gpg] https://mkvtoolnix.download/ubuntu/" $(lsb_release -cs) main > /etc/apt/sources.list.d/mkvtoolnix.download.list'
!sudo wget -q -O /usr/share/keyrings/gpg-pub-moritzbunkus.gpg https://mkvtoolnix.download/gpg-pub-moritzbunkus.gpg
!sudo apt -qq update
!sudo apt -qq install mkvtoolnix >/dev/null
!pip install pymkv
!mkvmerge -V

# **Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#**Navigate to the folder containing the episodes**

*Follow the Menu till you reach the required folder,
If there's no input option, then just stop and run the cell again*

**Choose drive first, followed by either "shared drives" or "my drive" and navigate to the desired folder**



In [ ]:
import os
import re
import subprocess
from pathlib import Path
from IPython.display import clear_output
import pymkv

current_directory = (Path.cwd()).resolve()

while True:
    items = sorted(list(current_directory.iterdir()))
    print("Current Directory: ", current_directory)
    for index, item in enumerate(items, 1):
        print('{0:2d}:'.format(index), item.name, end="\n")
  
    choice = int(input("Enter the choice (0 to stop, -1 for previous directory): "))

    if choice == 0:
        break
    
    if choice == -1:
        current_directory = current_directory.parent
    else:
        current_directory = current_directory / items[choice - 1].name

    os.chdir(current_directory)
    clear_output()

clear_output()
print("Current Directory: ", current_directory)

# **Shell commands to extract and mux .mks files**

In [ ]:
# Shell command to extract fonts, chapters etc.
!for f in *.mkv; do mkvmerge -o "output/${f%.mkv}.mks" --no-audio --no-video "${f%}"; done

# Shell command to mux in the .mks files
!for f in *.mkv; do mkvmerge -o "output/${f%}" "${f%}" "${f%.mkv}.mks"; done

# **Testing Building Command**
This one for Hyouka UDF

In [ ]:
from pathlib import Path
import os

mkvfiles = [x for x in Path.cwd().iterdir() if x.suffix == '.mkv']
attachments = [y for y in (Path.cwd()/'attachments').iterdir()]

commands = []

for i, mkvfile in enumerate(mkvfiles):
	command = ""

	command += f"mkvmerge --output \"output/{mkvfile.name}\" \"{mkvfile.name}\" "

	attach_folder = attachments[i]
	subtitles = [s for s in attach_folder.iterdir() if s.suffix == ".ass"]
	folder = [f for f in attach_folder.iterdir() if f.is_dir()]

	for sub in subtitles:
		if 'track05' in sub.name.lower():
			command += f"--language 0:en --track-name \"0:Full Subtitles (Tsundere+Commie)\" --default-track 0:yes \"{sub.resolve().as_posix()}\" "
		elif 'track07' in sub.name.lower():
			command += f"--language 0:en --track-name \"0:Full Subtitles (CoalGirls)\" \"{sub.resolve().as_posix()}\" "
		elif 'track09' in sub.name.lower():
			command += f"--language 0:en --track-name \"0:Full Subtitles (Mazui)\" \"{sub.resolve().as_posix()}\" "

	fcount = 0
	fold = folder[0].iterdir()
	for font in fold:
		command += f"--attachment-name \"{font.name}\" "
		if font.suffix.lower in ['.ttf', '.ttc']:
			command += "--attachment-mime-type application/x-truetype-font "
			fcount += 1
		elif font.suffix.lower == '.otf':
			command += "--attachment-mime-type application/vnd.ms-opentype "
			fcount += 1

		command += f"--attach-file \"{font.resolve().as_posix()}\" "

	command += "--track-order 0:0,0:1,1:0,2:0,3:0"

	if fcount == len(list(fold)):
		print(i + 1)
		commands.append(command)

with open("commands.txt", 'w') as f:
	for c in commands:
		f.write(c)
		f.write('\n\n')

In [ ]:
for item in sorted((Path.cwd() / "output").iterdir()):
    print(item.name)
    output = subprocess.run(f"7z h \"{item}\"", shell=True, capture_output=True)
    print(output.stdout.decode())

#**Batch Purge**

Works on the mkvfiles present in the current directory


In [ ]:
# Purges tracks based on user input
current_directory = (Path.cwd()).resolve()
output_directory = (current_directory / 'output').resolve()

def create_folder():
	'''Creates the output folder if it doesn't already exist'''

	if not output_directory.exists():
		output_directory.mkdir()
	

def demux(mkv, mkvfile, count):

	if count:
		for _ in range(count):
			mkv.remove_track(0)
		mkv.mux(output_directory / mkvfile, silent=True)
	else:
		print('No Tracks To Remove')


def display_track(track, i, err=''):
	'''Displays the tracks for a given mkvfile'''

	if err != '':
		print('{:<3} {:<10} {:<10} {:<25} {:<25}'.format(i, track.track_type, track.language, str(track.track_name), err))
	else:

		print('{:<3} {:<10} {:<10} {:<25}'.format(i, track.track_type, track.language, str(track.track_name)))


def remove(mkv, tracks, choice):
	'''Chooses the tracks to remove from the mkvfile'''

	print('\nTracks Present:')
	print('\n{:<3} {:<10} {:<10} {}\n----------------------------------------------'.format('ID', 'Type', 'Language', 'Name'))

	for i, track in enumerate(tracks, 1):
		display_track(track, i)
	print('\n')

	count = 0
	TID = -1
	errors = []

	print('\nRemoving Tracks:')
	print('\n{:<3} {:<10} {:<10} {}\n----------------------------------------------'.format('ID', 'Type', 'Language', 'Name'))
	for track in tracks:
		TID = -1
		try:
			if ('1' in choice) and ((track.track_type == 'audio' and track.language not in ['jpn', 'und']) or \
				(track.track_type == 'subtitles' and track.language not in ['eng', 'und', 'jpn']) or \
				('sign' in track.track_name.lower()) or ('eng' in track.track_name.lower() and track.track_type == 'audio')):

				TID = track.track_id


			if ('2' in choice) and ('commentary' in track.track_name):
				TID = track.track_id

			if TID != -1:
				count += 1
				display_track(track, TID + 1)
				mkv.move_track_front(TID)

		except Exception as e:
			errors.append((track, e))
			continue

	if errors:
		print('\nErrors while parsing, these tracks were skipped from being purged')
		print('\n{:<3} {:<10} {:<10} {:<25} {}\n----------------------------------------------'.format('ID', 'Type', 'Language', 'Name', 'Err Msg'))
		for track, error in errors:
			display_track(track, track.track_id + 1, err=str(error))


	return count		


def main():
	create_folder()
	mkvfiles = sorted([item for item in current_directory.iterdir() if item.suffix == '.mkv' and not item.is_dir()])

	print('Listing all the files:')
	for i, mkvfile in enumerate(mkvfiles, 1):
		print('{:<2} - {}'.format(i, mkvfile.name))

	choice = input('1) Remove Only Dub\n2) Remove Only Commentary\nChoice: ')

	if choice not in  ['1', '2', '21', '12']:
		exit(0)

	for i, mkvfile in enumerate(mkvfiles, 1):
		print('\n\n----------------------------------------------')
		print('{:<3} Working File: '.format(i), mkvfile.name, end='\n')
		print('----------------------------------------------')

		try:
			mkv = pymkv.MKVFile(mkvfile)
		except KeyError as e:
			print('ERROR:', e, 'NOT RECOGNIZED, SKIPPING CURRENT FILE')
			continue

		mkv.no_global_tags()
		tracks = mkv.get_track()

		count = remove(mkv, tracks, choice)
		demux(mkv, mkvfile.name, count)

	print('\nDone')


if __name__ == '__main__':
	main()
	

#**Batch Extract (Not Recommended, use the shell commands instead)**

Works on the mkvfiles present in the current directory

In [ ]:
# Extracts all the tracks, chapters, tags and fonts
current_directory = (Path.cwd()).resolve()
output_directory = (current_directory / 'output').resolve()

def create_folder():
	'''Creates the output folder if it doesn't already exist'''

	if not output_directory.exists():
		output_directory.mkdir()
		

def extract_fonts(mkvfile):
	'''Extracts the fonts from the mkvfile'''

	fonts = []
	fonts_command = ''

	mkvinfo = subprocess.run('mkvmerge --identify \"{}\"'.format(mkvfile), capture_output=True, text=True, shell=True)
	pattern = re.compile(r'name \'.+(.ttf|.otf|.TTF|.OTF)\'')
	matches = pattern.finditer(mkvinfo.stdout)

	for match in matches:
		fonts.append(mkvinfo.stdout[match.start() + 6 : match.end() - 1])

	attachment_folder = (output_directory / mkvfile.name).resolve()
	font_folder = (attachment_folder / 'attachments').resolve()
	
	if not attachment_folder.exists():		
		attachment_folder.mkdir()
		font_folder.mkdir()

	for i, font in enumerate(fonts, 1):
		fonts_command += '{}:\"{}\" '.format(i, font_folder / font)
	
	if fonts_command != '':
		subprocess.run('mkvextract \"{}\" attachments {}'.format(mkvfile, fonts_command), shell=True)
	else:
		print('No fonts\n')


def extract_subs(mkvfile):
	'''Extracts the subs from the mkvfile'''

	mkv = pymkv.MKVFile(mkvfile)
	tracks = mkv.get_track()

	subtitle_command = ''

	for track in tracks:
		if track.track_type == 'subtitles':
			subtitle_command += '{}:\"{}/track{}_{}.ass\" '.format(track.track_id, output_directory / mkvfile.name, track.track_id + 1,  track.language)

	if subtitle_command != '':
		subprocess.run('mkvextract \"{}\" tracks {}'.format(mkvfile, subtitle_command), shell=True)
	else:
		print('No subtitles\n')


def extract_chapters(mkvfile):
	'''Extracts the chapter from the mkvfile'''

	subprocess.run('mkvextract \"{}\" chapters \"{}\"'.format(mkvfile, output_directory / mkvfile.name / 'chapters.xml'), shell=True)


def main():
	create_folder()
	mkvfiles = sorted([item for item in current_directory.iterdir() if item.suffix == '.mkv'])

	if mkvfiles:
		for mkvfile in mkvfiles:
			print('Working File: ', mkvfile.name, end='\n\n')

			print('\nExtracting Fonts:\n')
			extract_fonts(mkvfile)

			print('\nExtracting Chapters:\n')
			extract_chapters(mkvfile)

			print('\nExtracting Subs:\n')
			extract_subs(mkvfile)

			print("\n")


if __name__ == '__main__':
	main()

#**Batch Mux**

This script is used to batch append Chapters, fonts and subtitles into the mkvfiles.\
Follows the animetosho attachments download folder structure.\
(Created for anime use)

```
-------Folder Structure-------

.
├── append
│   ├── [BlurayDesuYo] Shingeki no Kyojin (The Final Season) - 60 (BD 1920x1080 x265 10bit FLAC) [F97B223C].mkv
│   │   ├── attachments
│   │   │   ├── AGARAMONDPRO-REGULAR.OTF
│   │   │   ├── ANGIE-BOLDITALIC.TTF
│   │   │   ├── ANGIE-BOLD.TTF
│   │   │   ├── BSOD.TTF
│   │   │   ├── CLEARFACESSIBOLD.TTF
│   │   │   ├── GAR-A-MONDTALL-LIGHT.TTF
│   │   │   └── KINESISSTD-BLACKITALIC.OTF
│   │   ├── chapters.xml
│   │   ├── tags.xml
│   │   └── track3_eng.ass
│   └── [BlurayDesuYo] Shingeki no Kyojin (The Final Season) - 61 (BD 1920x1080 x265 10bit FLAC) [93A2A975].mkv
│       ├── attachments
│       │   ├── AGARAMONDPRO-REGULAR.OTF
│       │   ├── ANGIE-BOLDITALIC.TTF
│       │   ├── ANGIE-BOLD.TTF
│       │   ├── BSOD.TTF
│       │   ├── CLEARFACESSIBOLD.TTF
│       │   ├── GAR-A-MONDTALL-LIGHT.TTF
│       │   └── KINESISSTD-BLACKITALIC.OTF
│       ├── chapters.xml
│       ├── tags.xml
│       └── track3_eng.ass
├── mkvfiles
│   ├── [Kawaiika-Raws] Shingeki no Kyojin (2020) 01 [BDRip 1920x1080 HEVC FLAC].mkv [CDBF88C6]
│   └── [Kawaiika-Raws] Shingeki no Kyojin (2020) 02 [BDRip 1920x1080 HEVC FLAC].mkv [42248FF4]
└── output (will be created)
    ├── [Kawaiika-Raws] Shingeki no Kyojin (2020) 01 [BDRip 1920x1080 HEVC FLAC].mkv [E5B40389]
    └── [Kawaiika-Raws] Shingeki no Kyojin (2020) 02 [BDRip 1920x1080 HEVC FLAC].mkv [9EFD583F]
```

The above folder structure is to be followed.

In [ ]:
# Muxes subs, attachments and chapters into a mkvfile
parent_directory = (Path.cwd()).resolve()
mkvfiles_directory = (parent_directory / 'mkvfiles').resolve()
attachments_directory = (parent_directory / 'append').resolve()
output_directory = (parent_directory / 'output').resolve()

def create_folder():
	'''Creates the output folder if it doesn't already exist'''

	if not output_directory.exists():
		output_directory.mkdir()

	if not attachments_directory.exists():
		attachments_directory.mkdir()


def prerequisite():

	print("\nChoose the properties to ignore which are already present in the file (Remove properties in the final output)")
	print("--1) Chapters\n--2) Attachments (Fonts)\n--3) Global Tags\n--4) Track Tags")
	existing = input("Enter the numbers (Enter 0 to skip this menu): ")

	print("\nChoose the properties to ignore which are to be appended to the file (Won't append to the final output)")
	print("--1) Chapters\n--2) Attachments (Fonts)\n--3) Subtitles")
	insert = input("Enter the numbers (Enter 0 to skip this menu, adds all the files present in the attachments folder): ")

	links = []
	print('\nEnter 0 to skip the next menu if "append" folder is already populated')
	while True:
		link = input('Enter download link (Enter 0 to stop): ')
		if link == "0":
			break
		
		links.append(link)
  
	if links:
		print('Downloading...')
		for link in links:
			subprocess.run('wget "{}" -P "{}"'.format(link, parent_directory), shell=True)

		print('Extracting...')
		for file in parent_directory.iterdir():
			if file.suffix in ['.7z', '.zip']:
				subprocess.run('7z x "{}" -o"{}"'.format(file.name, attachments_directory), shell=True)

	return [existing, insert]


def get_list(source_directory):
	'''Returns a list of items present in a directory specified by the parameter'''

	temp_list = [item for item in source_directory.iterdir()]
	return sorted(temp_list)


def append(attachments_folder, mkv, insert):
	'''Appends the fonts, subtitle tracks and chapters if present'''

	font_folders = []
	fonts_present = []
	subs = []
	chapter = None

	for item in attachments_folder.iterdir():
		if item.is_dir() and '2' not in insert:
			font_folders.append(item)
		elif item.suffix == '.ass' and '3' not in insert:
			subs.append(item)
		elif ('chapters' in item.name.lower() and item.suffix == '.xml') and '1' not in insert:
			chapter = item
		else:
			continue
	
	filename = mkv.tracks[0]._file_path.split('\\')[-1]
	mkvmerge_json = subprocess.run(["mkvmerge", "-J", f"\"{filename}\""], capture_output=True, text=True, shell=True)
	fonts = mkvmerge_json["attachments"]
	if font_folders:
		print('\nAdding fonts:')
		for font_folder in font_folders:
			for font in font_folder.iterdir():
				if font not in fonts:
					print('--{}'.format(font.name))
					mkv.add_attachment(str(font))
	else:
		print('\nNo fonts to append')

	if subs:
		print('\nAdding subtitles:')
		for subtitle in subs:
			print('--{}'.format(subtitle.name))
			mkv.add_track(str(subtitle))
	else:
		print('\nNo subtitles to append')

	if chapter:
		print('\nAdding chapter:\n--{}'.format(chapter.name))
		mkv.chapters(str(chapter))
	else:
		print('\nNo chapters to append')


def muxing(mkv, mkvfile):
	'''Muxes the mkv file with the changes made'''

	mkv.mux(output_directory / mkvfile.name, silent=True)
	print('\nDone.')


def ignore_existing(mkv, choice):
	'''Ignore certain MKV file properties which are present in the file'''

	if '0' in choice:
		return

	print('\nIgnoring the following properties present in the file')
	if '1' in choice:
		print('--chapters')
		mkv.no_chapters()
	if '2' in choice:
		print('--attachments')
		mkv.no_attachments()
	if '3' in choice:
		print('--global tags')
		mkv.no_global_tags()
	if '4' in choice:
		print('--track tags')		
		mkv.no_track_tags()


def main():
	create_folder()
	existing, insert = prerequisite()
	mkvfiles = get_list(mkvfiles_directory)
	attachments = get_list(attachments_directory)

	for i, mkvfile in enumerate(mkvfiles):
		print('---------------------------------------------------------------------------')
		print('\nWorking File: ', mkvfile.name)
		mkv = pymkv.MKVFile(mkvfile)
		ignore_existing(mkv, existing)
		append(attachments[i], mkv, insert)
		muxing(mkv, mkvfile)


if __name__ == '__main__':
	main()
